# 2-Dataset

到这里我们便完成了对于 MiniMind Tokenizer 和 Model 部分的全部了解，我们所熟悉的大语言模型正是由这个组件构成的，接下来，我们需要对大模型训练所使用的数据集结构有个基本的认识。

想要训练一个能够正常对话，并且符合人类对话偏好的大模型一般需要经过以下几个训练阶段：

- 预训练（Pre-training）
- 有监督微调（Supervised Fine-tuning，SFT）
- 人类反馈强化学习（Reinforcement Learning from Human Feedback，RLHF）

在不同训练阶段使用的数据集有所不同，下面会从 MiniMind 代码出发进行介绍和解读。

In [2]:
import json
import random
import re

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
import os
import ast
from transformers import AutoTokenizer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/hey/anaconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 从 ../model 目录加载分词器

tokenizer = AutoTokenizer.from_pretrained('../model/minimind_tokenizer')
print(tokenizer.vocab_size)

6400


## 预训练数据集

预训练是模型在大规模语料上进行无监督学习的训练阶段，在该阶段，模型主要学习下一词预测的能力，简单的来说就是学会说话，而不是胡言乱语。因此，该阶段训练的模型不会具有问答能力，而是根据用户输入进行简单的词语接龙。

我们可以看一看预训练的数据集格式：

```
{"text": "如何才能摆脱拖延症？ 治愈拖延症并不容易，但以下建议可能有所帮助..."}
```

为了降低该 demo 的运行门槛，在 `./demo` 文件夹下提供了包含两条训练数据的 `pretrain_data.jsonl` 文件作为熟悉训练流程的数据集 demo。

In [4]:
# 我们可以查看一下 demo 中提供的数据
path_pretrain = './toydata/pretrain_data.jsonl'

with open(path_pretrain, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'text': 'LLM首先要学习的并非直接与人交流，而是让网络参数中充满知识的墨水，“墨水” 理论上喝的越饱越好，产生大量的对世界的知识积累。 预训练就是让Model先埋头苦学大量基本的知识，例如从Wiki百科、新闻、书籍整理大规模的高质量训练数据。 这个过程是“无监督”的，即人类不需要在过程中做任何“有监督”的校正，而是由模型自己从大量文本中总结规律学习知识点。 模型此阶段目的只有一个：学会词语接龙。例如我们输入“秦始皇”四个字，它可以接龙“是中国的第一位皇帝”。'}

Row 2: {'text': '经过预训练，LLM此时已经掌握了大量知识，然而此时它只会无脑地词语接龙，还不会与人聊天。 SFT阶段就需要把半成品LLM施加一个自定义的聊天模板进行微调。 例如模型遇到这样的模板【问题->回答，问题->回答】后不再无脑接龙，而是意识到这是一段完整的对话结束。 称这个过程为指令微调，就如同让已经学富五车的「牛顿」先生适应21世纪智能手机的聊天习惯，学习屏幕左侧是对方消息，右侧是本人消息这个规律。 在训练时，MiniMind的指令和回答长度被截断在512，是为了节省显存空间。就像我们学习时，会先从短的文章开始，当学会写作200字作文后，800字文章也可以手到擒来。 在需要长度拓展时，只需要准备少量的2k/4k/8k长度对话数据进行进一步微调即可（此时最好配合RoPE-NTK的基准差值）。'}



我们知道，构建一个深度学习数据集需要继承 `torch.utils.data.dataset`，并构建 DataLoader 数据迭代器进行迭代访问。下面，我们来看看 MiniMind 是如何抽象一个预训练数据集的。

In [5]:
class PretrainDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(data_path)

    def load_data(self, path):
        """按行读取 jsonl 文件，并存储在列表中"""
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        sample = self.samples[index]

        text = f"{self.tokenizer.bos_token}{str(sample['text'])}{self.tokenizer.eos_token}"
        print(text) # uncomment to see formating prompt
        encoding = self.tokenizer(
            text,
            max_length=self.max_length, # 512
            padding='max_length', # 填充到512 # 'longest' 填充到batch中最长序列
            truncation=True, # 截断
            return_tensors='pt' # 返回tensor
        )
        print(encoding) # uncomment to see encoding result
        input_ids = encoding.input_ids.squeeze() # 压缩维度 # {'input_ids': tensor([[1, 2, 3, 4]])} -> tensor([1, 2, 3, 4])
        loss_mask = (input_ids != self.tokenizer.pad_token_id) # mask to ignore loss on pad token

        # 嵌入层要求输入是整数类型（torch.long），因为它是通过索引查找对应的向量。若使用浮点类型会直接报错。
        # 交叉熵损失（nn.CrossEntropyLoss）的目标（Y）必须是整数类型，表示类别索引
        # loss_mask设为torch.long是历史习惯或兼容性处理，但更推荐使用torch.bool或torch.float以提高代码可读性。
        X = torch.tensor(input_ids[:-1], dtype=torch.long) # <eos> token not included # torch.long 即 int64
        Y = torch.tensor(input_ids[1:], dtype=torch.long) # <bos> token not included
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long) # align to tensor X
        return X, Y, loss_mask

In [6]:
pretrain_dataset = PretrainDataset(path_pretrain, tokenizer)
print(f'预训练数据集长度{len(pretrain_dataset)}')
x, y, lm = pretrain_dataset[0]
print(x.shape, y.shape, lm.shape)

预训练数据集长度2
<s>LLM首先要学习的并非直接与人交流，而是让网络参数中充满知识的墨水，“墨水” 理论上喝的越饱越好，产生大量的对世界的知识积累。 预训练就是让Model先埋头苦学大量基本的知识，例如从Wiki百科、新闻、书籍整理大规模的高质量训练数据。 这个过程是“无监督”的，即人类不需要在过程中做任何“有监督”的校正，而是由模型自己从大量文本中总结规律学习知识点。 模型此阶段目的只有一个：学会词语接龙。例如我们输入“秦始皇”四个字，它可以接龙“是中国的第一位皇帝”。</s>
{'input_ids': tensor([[   1,   46,   46,   47, 2105,  404, 1511,  269,  581, 1003, 4656,  850,
          415, 3303,  270,  763,  345,  929, 3209, 5832,  413, 3708, 3095,  269,
          742,  104,  872,  270, 1024,  742,  104,  872, 1025,  223, 4680,  568,
         1080,  254,  269, 1883, 1135,  112, 1883,  587,  270, 2268, 3630,  269,
          510, 2219, 6371, 1456, 1591,  825,  110,  286,  223, 1619, 2875, 3541,
          929,   47,  565,  392, 1404,  675,  236, 2135, 6301,  595, 3630, 2885,
         6371, 1456,  270, 1373, 1083,   57, 3907,   75, 5466, 1493,  341, 5683,
          341, 2170, 6357, 1354,  578,  611, 1379, 1605,  269,  654, 1837, 2875,
         1327,  286,  223,  990, 1629,  345, 1024, 1308, 3608,  878,   99, 1025,
          26

/tmp/ipykernel_10254/3782234284.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long) # <eos> token not included # torch.long 即 int64
/tmp/ipykernel_10254/3782234284.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long) # <bos> token not included
/tmp/ipykernel_10254/3782234284.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long) # align to tensor X


## 有监督微调数据集

有监督微调（Supervised Fine Tuning，SFT）对预训练后得到的基座 LLM 施加一个自定义聊天模板进行微调，由于在这一阶段，模型训练的目标是根据用户指令生成响应（构建问答体系），故又称为指令微调。

我们可以看一看有监督微调的数据集格式：

```
{
    "conversations": [
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "你好！"},
        {"role": "user", "content": "再见"},
        {"role": "assistant", "content": "再见！"}
    ]
}
```

为了降低该 demo 的运行门槛，在 `./demo` 文件夹下提供了包含两条 conversation 问答数据的 `sft_data.jsonl` 文件作为熟悉训练流程的数据集 demo。

In [7]:
# 我们可以查看一下 demo 中提供的数据
path_sft = './toydata/sft_data.jsonl'

with open(path_sft, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'conversations': [{'role': 'user', 'content': '你好吗？'}, {'role': 'assistant', 'content': '我很好，谢谢！你呢？'}, {'role': 'user', 'content': '我也很好，谢谢！'}, {'role': 'assistant', 'content': '太好了！祝你今天愉快！'}]}

Row 2: {'conversations': [{'role': 'user', 'content': '你喜欢什么运动？'}, {'role': 'assistant', 'content': '我喜欢跑步和游泳。你呢？'}, {'role': 'user', 'content': '我喜欢打篮球！'}, {'role': 'assistant', 'content': '篮球很棒！是一个很好的团队运动。'}]}



接下来，我们尝试构造一个数据集对象，实现对 sft 格式数据的读取与处理。

In [11]:
# text转token
# add_special_tokens：自动添加模型预定义的特殊标记
'''
常见的特殊标记：
BERT 系列：[CLS]（分类标记，放在开头）、[SEP]（分隔句子或结尾）
GPT 系列：<s>（开始）、</s>（结束）
其他模型：如<pad>（填充）、<mask>（掩码）、<unk>（未知词）等
'''

print(tokenizer('<s>assistant\n', add_special_tokens=True).input_ids)
print(tokenizer('<s>assistant\n', add_special_tokens=False).input_ids)

[1, 1078, 538, 501, 201]
[1, 1078, 538, 501, 201]


In [27]:
class SFTDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(jsonl_path)
        self.bos_id = tokenizer('<s>assistant\n', add_special_tokens=False).input_ids # set bos token
        self.eos_id = tokenizer('</s>\n', add_special_tokens=False).input_ids # set eos token

    def load_data(self, path):
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip())
                samples.append(data)
        return samples
        
    def __len__(self):
        return len(self.samples)

    def _create_chat_prompt(self, conversations):
        """构建符合 ChatML 格式的对话"""
        messages = []
        for i, turn in enumerate(conversations): # for each speaker in one conversation
            role = 'user' if i % 2 == 0 else 'assistant'
            messages.append({"role": role, "content": turn['content']})
        return self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

    def _generate_loss_mask(self, input_ids):
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id: # check if reach bos token
                ########### find content start & end point ###########
                start = i + len(self.bos_id) # set start point at the end of bos token
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id: # check if reach eos token
                        break
                    end += 1
                ######################################################
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)): # ignore tokens that reach max input length
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids) # update i to exit current conversation turn
            else:
                i += 1
        return loss_mask
    
    def __getitem__(self, index):
        sample = self.samples[index]
        print("sample: ", sample)
        prompt = self._create_chat_prompt(sample['conversations'])
        print("prompt: ", prompt)
        # print(prompt) # uncomment to see formating prompt
        tokens = self.tokenizer.tokenize(prompt)
        # ids = self.tokenizer.convert_tokens_to_string
        input_ids = self.tokenizer(prompt).input_ids[:self.max_length] # encode input
        input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids)) # encode padding
        # print(input_ids) # uncomment to see encoded prompt
        
        loss_mask = self._generate_loss_mask(input_ids)

        X = torch.tensor(input_ids[:-1], dtype=torch.long)
        Y = torch.tensor(input_ids[1:], dtype=torch.long)
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long) # align with pred pos

        print("loss_mask=1所对应的text:")
        text = []
        for i, j in zip(tokens[:self.max_length-1], loss_mask):
            if j:
                text.append(i)
        print(self.tokenizer.convert_tokens_to_string(text))

        return X, Y, loss_mask

In [28]:
sft_dataset = SFTDataset(path_sft, tokenizer)
print(len(sft_dataset))
x, y, lm = sft_dataset[0]
print(f'样本 shape = {x.shape}, 标签 shape = {y.shape}, loss_mask shape {lm.shape}')
# print(lm) # 打印 loss mask，你会发现在序列中有两处以 1 填充的序列，这是因为我们在一个 conversation 中开展了两轮对话，其中只有 assistant 回复计算损失

2
sample:  {'conversations': [{'role': 'user', 'content': '你好吗？'}, {'role': 'assistant', 'content': '我很好，谢谢！你呢？'}, {'role': 'user', 'content': '我也很好，谢谢！'}, {'role': 'assistant', 'content': '太好了！祝你今天愉快！'}]}
prompt:  <s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
我很好，谢谢！你呢？</s>
<s>user
我也很好，谢谢！</s>
<s>assistant
太好了！祝你今天愉快！</s>

loss_mask=1所对应的text:
我很好，谢谢！你呢？</s>
太好了！祝你今天愉快！</s>

样本 shape = torch.Size([511]), 标签 shape = torch.Size([511]), loss_mask shape torch.Size([511])


## 人类反馈强化学习数据集

在 MiniMind 项目中，采用直接偏好优化（Direct Preference Optimization，DPO）训练大模型对齐人类偏好。在这一训练阶段，模型将会根据提供的问答正反例进行偏好优化，从而降低让人类不满意的答案出现的几率。

与PPO(Proximal Policy Optimization)这种需要奖励模型、价值模型的RL算法不同； DPO通过推导PPO奖励模型的显式解，把在线奖励模型换成离线数据，Ref模型输出可以提前保存。 DPO性能几乎不变，只用跑 actor_model 和 ref_model 两个模型，大大节省显存开销和增加训练稳定性。

我们可以看一看DPO的数据集格式：

```
{
  "chosen": [
    {"content": "Query", "role": "user"}, 
    {"content": "good answer", "role": "assistant"}
  ], 
  "rejected": [
    {"content": "Query", "role": "user"}, 
    {"content": "bad answer", "role": "assistant"}
  ]
}
```

为了降低该 demo 的运行门槛，在 ./demo 文件夹下提供了包含两条 问答正反例 数据的 dpo_data.jsonl 文件作为熟悉训练流程的数据集 demo。

In [34]:
# 我们可以查看一下 demo 中提供的数据
path_dpo = './toydata/dpo_data.jsonl'

with open(path_dpo, 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        data = json.loads(line.strip())
        print(f'Row {line_num}: {data}\n')

Row 1: {'chosen': [{'content': '你好吗？', 'role': 'user'}, {'content': '我很好，谢谢！你呢？', 'role': 'assistant'}, {'content': '今天过得怎么样？', 'role': 'user'}, {'content': '挺好的，去跑步了，心情不错。', 'role': 'assistant'}], 'rejected': [{'content': '你好吗？', 'role': 'user'}, {'content': '不好，我很累。', 'role': 'assistant'}, {'content': '你喜欢什么运动？', 'role': 'user'}, {'content': '我不喜欢运动，没兴趣。', 'role': 'assistant'}]}

Row 2: {'chosen': [{'content': '你喜欢旅行吗？', 'role': 'user'}, {'content': '我喜欢，尤其是去海边。', 'role': 'assistant'}, {'content': '你去过哪些地方？', 'role': 'user'}, {'content': '我去过欧洲、美国和日本，都是很棒的经历。', 'role': 'assistant'}], 'rejected': [{'content': '你喜欢旅行吗？', 'role': 'user'}, {'content': '不太喜欢，觉得太麻烦。', 'role': 'assistant'}, {'content': '你去过哪些地方？', 'role': 'user'}, {'content': '没去过太多地方，主要是待在家里。', 'role': 'assistant'}]}



```json
{
    'chosen': [
        {'content': '你好吗？', 'role': 'user'}, 
        {'content': '我很好，谢谢！你呢？', 'role': 'assistant'}, 
        {'content': '今天过得怎么样？', 'role': 'user'}, 
        {'content': '挺好的，去跑步了，心情不错。', 'role': 'assistant'}
    ], 
    'rejected': [
        {'content': '你好吗？', 'role': 'user'}, 
        {'content': '不好，我很累。', 'role': 'assistant'}, 
        {'content': '你喜欢什么运动？', 'role': 'user'}, 
        {'content': '我不喜欢运动，没兴趣。', 'role': 'assistant'}
    ]}
```
```json
{
    'chosen': [
        {'content': '你喜欢旅行吗？', 'role': 'user'}, 
        {'content': '我喜欢，尤其是去海边。', 'role': 'assistant'}, 
        {'content': '你去过哪些地方？', 'role': 'user'}, 
        {'content': '我去过欧洲、美国和日本，都是很棒的经历。', 'role': 'assistant'}
    ], 
    'rejected': [
        {'content': '你喜欢旅行吗？', 'role': 'user'}, 
        {'content': '不太喜欢，觉得太麻烦。', 'role': 'assistant'}, 
        {'content': '你去过哪些地方？', 'role': 'user'}, 
        {'content': '没去过太多地方，主要是待在家里。', 'role': 'assistant'}
    ]}
```

接下来，我们尝试构造 json 对象，实现对 dpo 格式数据的读取和处理

In [35]:
class DPODataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=512):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.padding = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        self.bos_id = tokenizer('<s>assistant\n', add_special_tokens=False).input_ids # content prefix
        self.eos_id = tokenizer('</s>\n', add_special_tokens=False).input_ids # content suffix
        with open(file_path, 'r', encoding='utf-8') as f: # load data
            self.data = []
            for line in f:
                line = line.strip()
                obj = json.loads(line)
                self.data.append(obj)

    def __len__(self):
        return len(self.data)

    def _generate_loss_mask(self, input_ids):
        """此处的损失掩码生成函数与 SFT 阶段逻辑一致"""
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id)
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)):
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask
    
    def __getitem__(self, index):
        item = self.data[index]
        chosen = item['chosen'] # 一个 list，里面包含若干 {role, content}
        rejected = item['rejected'] # 同上
        # format prompt
        chosen_prompt = self.tokenizer.apply_chat_template(
            chosen, tokenize=False, add_generation_prompt=False
        )
        print(chosen_prompt) # uncomment to see formating prompt
        rejected_prompt = self.tokenizer.apply_chat_template(
            rejected, tokenize = False, add_gerneration_prompt=False
        )
        print(rejected_prompt) # uncomment to see formating prompt
        # tokenize
        chosen_encoding = self.tokenizer(
            chosen_prompt, truncation=True, max_length=self.max_length, padding='max_length'
        )
        rejected_encoding = self.tokenizer(
            rejected_prompt, truncation=True, max_length=self.max_length, padding='max_length'
        )
        # generate loss mask
        chosen_input_ids = chosen_encoding['input_ids']
        chosen_loss_mask = self._generate_loss_mask(chosen_input_ids)
        rejected_input_ids = rejected_encoding['input_ids']
        rejected_loss_mask = self._generate_loss_mask(rejected_input_ids)
        # same as sft / pretrain
        x_chosen = torch.tensor(chosen_input_ids[:-1], dtype=torch.long)
        y_chosen = torch.tensor(chosen_input_ids[1:], dtype=torch.long)
        mask_chosen = torch.tensor(chosen_loss_mask[1:], dtype=torch.long)
        x_rejected = torch.tensor(rejected_input_ids[:-1], dtype=torch.long)
        y_rejected = torch.tensor(rejected_input_ids[1:], dtype=torch.long)
        mask_rejected = torch.tensor(rejected_loss_mask[1:], dtype=torch.long)

        return {
            'x_chosen': x_chosen,
            'y_chosen': y_chosen,
            'mask_chosen': mask_chosen,
            'x_rejected': x_rejected,
            'y_rejected': y_rejected,
            'mask_rejected': mask_rejected
        }

In [36]:
dpo_dataset = DPODataset(path_dpo, tokenizer)
print(f'DPO 数据集长度：{len(dpo_dataset)}')
res = dpo_dataset[0]
# 如有需要，请自主查看 res 中的元素

DPO 数据集长度：2
<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
我很好，谢谢！你呢？</s>
<s>user
今天过得怎么样？</s>
<s>assistant
挺好的，去跑步了，心情不错。</s>

<s>system
你是 MiniMind，是一个有用的人工智能助手。</s>
<s>user
你好吗？</s>
<s>assistant
不好，我很累。</s>
<s>user
你喜欢什么运动？</s>
<s>assistant
我不喜欢运动，没兴趣。</s>

